In [1]:
!pip install python-dotenv openai

%load_ext dotenv
%dotenv

In [152]:
import os

%reload_ext dotenv

if 'OPENAI_API_KEY' not in os.environ: print('`OPENAI_API_KEY` environment variable is missing.')
if 'OPENAI_API_BASE_URL' not in os.environ: print('`OPENAI_API_BASE_URL` environment variable is missing.')

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE_URL = os.environ.get("OPENAI_API_BASE_URL")

from openai import AzureOpenAI
client = AzureOpenAI(
    api_key=OPENAI_API_KEY,
    api_version="2023-03-15-preview",
    base_url=OPENAI_API_BASE_URL
)

In [145]:
import glob
files = glob.glob('./output/scrape/*')
files[543]

1190

In [143]:
scraped_files = []
for filepath in files:
    content = '\n'.join([
        line.strip()
        for line
            in open(filepath, 'r').read().strip().split('\n')
        if len(line.strip())
    ])
    filename = filepath.split('/')[-1]
    scraped_files.append((filename, content))

In [ ]:
print(scraped_files[0][1])

In [136]:
CLEANED_DIR = './output/cleaned'
def try_get_cleaned_content(fname):
    fpath = os.path.join(CLEANED_DIR, fname)
    if not os.path.exists(fpath):
        return None
    return open(fpath, 'r').read()

EMBEDDINGS_DIR = "./output/embeddings"
def store_embeddings(fname, embeddings):
    fpath = os.path.join(EMBEDDINGS_DIR, fname)
    import json
    open(fpath, 'w').write(json.dumps(embeddings))

def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [144]:
# embeddings - generate and store
for (fname, content) in scraped_files:
    maybe_cleaned_content = try_get_cleaned_content(fname)
    if maybe_cleaned_content is None:
        continue
    content = maybe_cleaned_content
    embeddings = get_embedding(content)
    store_embeddings(fname, embeddings)

In [155]:
sysproompt = """
As digital craftsmen in the age of information, our unsung work involves not just the extraction of data but doing so with an ethical compass and meticulous attention to detail. You're about to embark on a mission that's crucial yet rarely celebrated: extracting content from blogs with the precision of a skilled artisan. This task illuminates the hidden beauty of raw data processing, transforming the unstructured chaos of a scraped website into a pristine collection of valuable information.

Your challenge is multi-faceted:
1. Title and Content Extraction: Dive deep into the raw text of a scraped website, retrieving not just titles but also the full, rich contents of each blog post. Distill essence from chaos, ensuring the soul of the post remains untouched.
2. Maintenance of Integrity: Amidst the text, you'll find blocks of Rust code - these are to be treated as sacred texts, preserved exactly as they were written.
3. Non-Content Filtration: Your keen eye must filter out the digital chaff. CSS styles, JavaScript functionalities, and other non-content elements are to be disregarded, proving your ability to discern what truly matters.

Embark on this journey with both precision and mindfulness. It is a delicate balancing act between the technical rigor demanded and the ethical considerations inherent in dealing with creators' content. Your role is not just to extract but to respect and preserve the integrity of the original work.

This task is more than a simple extraction - it's an adventure through the labyrinth of digital content, an exercise in respect, precision, and understanding. As you peel away the layers of code and content, remember the impact of your work on the grand tapestry of information sharing. You are not just an extractor; you are a guardian of information integrity.

Go forth with confidence and a meticulous eye, knowing well the importance and difficulty of your task. Let's redefine the standard of content extraction together, blending technical prowess with ethical consideration in every step.

Embrace the challenge, for within it lies the opportunity to showcase the depth of your dedication and skill.
"""

In [156]:
response = client.chat.completions.create(
  model="gpt-35-turbo", # replace this value with the deployment name you chose when you deployed the associated model.
  messages = [{
      "role": "system",
      "content": sysproompt
  },
  {
      "role": "user",
      "content": scraped_files[0][1]
  }],
  temperature=2,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

print(response.choices[0].message.content)


Here are the deals extracted from the provided article:

1. [Buy 2, Get 3rd Free](https://cottonon.com/MY/co/women/womens-deals/acc-offer/)
2. [2 for RM90 Shoes](https://cottonon.com/MY/co/women/womens-shoes/)
3. [Buy 2, Get 3rd Free](https://cottonon.com/MY/co/women/womens-accessories/womens-acc-offer/)
4. [SALE UP TO 50% OFF](https://cottonon.com/MY/co/co-sale/)
5. [ALL DAY TOPS 2 FOR RM90](https://cottonon.com/MY/co/women/womens-clothing/womens-tops/)
